# Create a Point to Point (Layer 2) Network with Explicit Route Options (ERO)

This notebook shows how to create an isolated Point 2 Point Layer2 Ethernet, specify Explicity Route Options (ERO) to control Quality of Service and connect compute nodes to it and use FABlib's automatic configuration functionality.

## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

The script below configures two nodes on the Tera Core, each equipped with a dedicated NIC connected to two separate WAN Ethernets, one with and one without Explicit Routes.

Two nodes are instantiated, each with a dedicated NIC component, as a Point-to-Point connection is required, which is only supported by dedicated NICs. The nodes are then connected via two distinct Wide Area Layer 2 networks, with each network following a different explicitly specified path. The resulting latency differences between the two paths are then observed.


NIC component model options include:
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports)
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports)

In [ ]:
slice_name = 'MySlice-ero2'
[site1, site2, site3, site4] = ["ATLA", "WASH", "STAR", "SALT"]
print(f"Sites: {site1}, {site2}, {site3}, {site4}")

node1_name = 'Node1'
node2_name = 'Node2'
# Path1 : LOSA -> SALT -> NEWY
# Path2 : LOSA -> DALL -> NEWY
net1_name = 'net-with-ero-path1' 
net2_name = 'net-with-ero-path2'

In [ ]:

#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=net1_name, subnet=IPv4Network("192.168.1.0/24"), type="L2PTP")

net2 = slice.add_l2network(name=net2_name, subnet=IPv4Network("192.168.2.0/24"), type="L2PTP")

# Node1
node1 = slice.add_node(name=node1_name, site=site1)
n1_nic1 = node1.add_component(model='NIC_ConnectX_5', name='nic1')
node1_iface1 = n1_nic1.get_interfaces()[0]
node1_ch_iface1 = node1_iface1.add_sub_interface("child1", vlan="100")
node1_ch_iface1.set_mode('auto')

node1_iface2 = n1_nic1.get_interfaces()[1]
node1_ch_iface2 = node1_iface2.add_sub_interface("child1", vlan="200")
node1_ch_iface2.set_mode('auto')

net1.add_interface(node1_ch_iface1)
net2.add_interface(node1_ch_iface2)

#n1_nic1.get_interfaces()[0].set_mode('auto')
#n1_nic1.get_interfaces()[0].set_vlan('100')
#n1_nic1.get_interfaces()[1].set_mode('auto')
#n1_nic1.get_interfaces()[1].set_vlan('200')

#net1.add_interface(n1_nic1.get_interfaces()[0])

#net2.add_interface(n1_nic1.get_interfaces()[1])

# Node2
node2 = slice.add_node(name=node2_name, site=site4)
n2_nic1 = node2.add_component(model='NIC_ConnectX_5', name='nic1')


node2_iface1 = n2_nic1.get_interfaces()[0]
node2_ch_iface1 = node2_iface1.add_sub_interface("child1", vlan="100")
node2_ch_iface1.set_mode('auto')

node2_iface2 = n2_nic1.get_interfaces()[1]
node2_ch_iface2 = node2_iface2.add_sub_interface("child1", vlan="200")
node2_ch_iface2.set_mode('auto')

net1.add_interface(node2_ch_iface1)
net2.add_interface(node2_ch_iface2)

#n2_nic1.get_interfaces()[0].set_mode('auto')
#n2_nic1.get_interfaces()[0].set_vlan('100')
#n2_nic1.get_interfaces()[1].set_mode('auto')
#n2_nic1.get_interfaces()[1].set_vlan('200')

#net1.add_interface(n2_nic1.get_interfaces()[0])

#net2.add_interface(n2_nic1.get_interfaces()[1])

# Set Explicit Route Options for Network1
net1.set_l2_route_hops(hops=[site2])

# Set Explicit Route Options for Network2
net2.set_l2_route_hops(hops=[site3])

#Submit Slice Request
slice.submit()

## Run the Experiment
We illustrate the variation in delay between the two networks, highlighting that latency varies depending on the selected paths.

In [ ]:
# Get Slice and Nodes
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)    

### Ping on the Network with Explicit Routes on Path1

In [ ]:
node2_net1_addr = node2.get_interface(network_name=net1_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_net1_addr}')

### Ping on the Network with Explicit Routes on Path2 

In [ ]:
node2_net2_addr = node2.get_interface(network_name=net2_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_net2_addr}')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()